<a href="https://colab.research.google.com/github/eiensatya7/ANN_projects/blob/main/miniproject_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem :

IMDB movie review sentiment classification problem. Each movie review is a variable sequence of words and the sentiment of each movie review must be classified. The IMDB Movie Review Dataset contains 25,000 highly-polar movie reviews (good or bad) for training and the same amount again for testing. The problem is to determine whether a given movie review has a positive or negative sentiment. Keras provides access to the IMDB dataset built-in. The imdb.load_data() function allows you to load the dataset in a format that is ready for use in neural network and deep learning models. The words have been replaced by integers that indicate the ordered frequency of each word in the dataset. The sentences in each review are therefore comprised of a sequence of integers.

# Why CNN with LSTM for text Classifcation

CNNs are generally used in computer vision, however they’ve recently been applied to various NLP tasks and the results were promising.
Let’s briefly see what happens when we use CNN on text data through a diagram.The result of each convolution will fire when a special pattern is detected. By varying the size of the kernels and concatenating their outputs, you’re allowing yourself to detect patterns of multiples sizes (2, 3, or 5 adjacent words).Patterns could be expressions (word ngrams?) like “I hate”, “very good” and therefore CNNs can identify them in the sentence regardless of their position.
Recurrent neural networks can obtain context information but the order of words will lead to bias; the text analysis method based on Convolutional neural network (CNN) can obtain important features of text through pooling but it is difficult to obtain contextual information which can be leverage using LSTM. So using the combination of CNN with LSTM could give us some intresting results

# Develop an text classification model based on CNN + LSTM in Keras.

In this assignment, you will have to train two Text classification:
1) LSTM based Text Classification
2) CNN + LSTM based Text Classification

After training the two different classification, you have to compare the F1 Score on both of the model trained and report the best F1 Score for which of them.

This notebook is divided into six parts. Total : [12 Marks]

1. Calculate the average length of reviews and modifying the length of sentences in X_train , X_test, X_cv [2 Mark]
2. Implement the LSTM model [3 Marks]
3. Calculate the LSTM model F1 Score [1 Mark]
4. Implement the CNN + LSTM [3 Marks]
5. Calculate the CNN + LSTM model F1 Score [1 Mark]
6. Identify and decode 5 of the sentences misclassified by LSTM model which  were correctly classified by CNN + LSTM model [2 Mark]

In [6]:
from sklearn.model_selection import train_test_split
from keras.datasets import imdb

from tensorflow.keras.preprocessing.sequence import pad_sequences  # to do padding or truncating
from tensorflow.keras.models import Sequential  # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense  # layers of the architecture

import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

#import the required library

# Student will have to code here


# Students will end their code here

In [7]:
# load the dataset but only keep the top n words, zero the rest
top_words = 10000

import numpy as np

np.load.__defaults__ = (None, True, True, 'ASCII')

# call load_data with allow_pickle implicitly set to true
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=10000)

X_train, X_cv, y_train, y_cv = train_test_split(X_train, y_train, test_size=0.2)
print("Shape of train data:", X_train.shape)
print("Shape of Test data:", X_test.shape)
print("Shape of CV data:", X_cv.shape)

print(X_train)



Shape of train data: (20000,)
Shape of Test data: (25000,)
Shape of CV data: (5000,)
[list([1, 260, 332, 2, 274, 13, 16, 1076, 685, 11, 4, 201, 21, 32, 11, 32, 12, 9, 179, 7180, 886, 4, 85, 795, 12, 9, 2, 8, 124, 15, 4, 2, 2, 26, 254, 33, 157, 319, 2, 23, 2, 2, 467, 175, 2, 10, 10, 9, 3643, 6, 117, 5648, 252, 40, 6, 176, 7, 2, 29, 1085, 27, 2604, 17, 4, 91, 674, 155, 126, 29, 1077, 4, 5263, 2, 2, 5, 2, 33, 1573, 175, 1434, 315, 4, 201, 75, 79, 12, 103, 44, 4, 86, 158, 234, 10, 10, 9, 3643, 6, 117, 4274, 11, 4, 201, 252, 4, 173, 44, 4, 2, 11, 1226, 938, 9, 572, 1139, 2, 49, 55, 196, 2732, 479, 180, 8, 2, 9092, 5, 2, 2, 56, 4, 226, 9182, 7, 1226, 938, 11, 44, 1119, 234, 21, 4, 213, 1287, 7513, 134, 183, 122, 11, 192, 6558, 21, 24, 484, 6771, 4, 1007, 18, 4, 2, 1023, 429, 4, 2921, 6714, 10, 10, 9, 29, 7656, 429, 1010, 4707, 11, 101, 96, 5898, 24, 6, 681, 24, 8, 61, 4889, 32, 29, 560, 9, 15, 1996, 256, 6, 1058, 173, 11, 2, 63, 6714, 4677, 53, 946, 24, 15, 1996, 9, 4, 64, 282, 10, 10, 11, 4

In [8]:
def get_avg_length(data):
    review_length = []
    largest_length = 0
    smallest_length = 1000000
    for review in data:
        length = len(review)
        largest_length = length if largest_length < length else largest_length
        smallest_length = length if smallest_length > length else smallest_length
        review_length.append(length)
    average_review_length = int(np.ceil(np.mean(review_length)))
    return average_review_length, largest_length, smallest_length


# truncate and pad input sequences
# Calculate the average length of reviews using the training set (X_train) and set the value to max_review_length
# truncate or pad the reviews so that length of all the reviews are same

average_review_length, largest_length, smallest_length = get_avg_length(X_train)

print(
    f"average review length {average_review_length}, smallest review length {smallest_length}, largest review length {largest_length}")

X_train = pad_sequences(X_train, maxlen=average_review_length, padding='post',
                        truncating='post')  # code to truncate or pad the reviews to the average_review_length
X_test = pad_sequences(X_test, maxlen=average_review_length, padding='post',
                       truncating='post')  # code to truncate or pad the reviews to the average_review_length
X_cv = pad_sequences(X_cv, maxlen=average_review_length, padding='post',
                     truncating='post')

print("Padded and truncated posts")

average_review_length, largest_length, smallest_length = get_avg_length(X_train)
print(
    f"average review length {average_review_length}, smallest review length {smallest_length}, largest review length {largest_length}")

# code to truncate or pad the reviews to the average_review_length

average review length 240, smallest review length 11, largest review length 2494
Padded and truncated posts
average review length 240, smallest review length 240, largest review length 240


In [9]:
# Decoding the data coded data of IMDB ( Data Understanding )
index = imdb.get_word_index()
reverse_index = dict([(value, key) for (key, value) in index.items()])
decoded = " ".join([reverse_index.get(i - 3, "#") for i in X_train[0]])
print(decoded)



# having read # book i was slightly disappointed in the series but all in all it is quite informative reading the other comments it is # to know that the # # are hard at work seeing # on # # under every # br br is diamond a little preachy sure like a lot of # he sees his theory as the most important thing ever he uses the phrase # # and # at seemingly every opportunity during the series we get it after about the first 10 minutes br br is diamond a little simplistic in the series sure the part about the # in south america is particularly amusing # some very long complicated history down to # swords and # # up the whole conquest of south america in about 15 minutes but the point remains valid these things did in fact contribute but not totally define the reasons for the # success against the established cultures br br is he preaching against western civilization in any way nope not a word not to my ear all he says is that luck played a large part in # which cultures advanced more quickly

In [10]:
# Architecture Diagram for LSTM Based Classifcation but you will have to change the
# configuration/model parameters while implementing it depending on the input , output and the 
# Problem statement.

from IPython.display import Image

#Image(filename='LSTM_model.png')


In [11]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

embedding_vector_length = 64
model_lstm = tf.keras.Sequential()

# Write the code for LSTM Based Classifcation
# Embedding layer
# LSTM Layer : You are free to choose the hyperparameters and the number of layers
# Dense Layer
# Use appropriate activation function in respective layers

# Students will be starting their code from here:

model_lstm.add(Embedding(10000, embedding_vector_length, input_length = average_review_length))
model_lstm.add(LSTM(64))
model_lstm.add(Dense(1, activation='sigmoid'))


checkpoint = ModelCheckpoint(
    'models/LSTM.h5',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)

# Students will be ending their code here

model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_lstm.summary())

# Change the number of epochs and the batch size depending on the RAM Size

#model_lstm.fit(X_train, y_train, epochs=5, batch_size=64, verbose=1, validation_data=(X_cv, y_cv))


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 240, 64)           6400      
_________________________________________________________________
lstm (LSTM)                  (None, 64)                33024     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 39,489
Trainable params: 39,489
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
# Final evaluation of the model using test dataset
# Students will be starting their code from here:



In [13]:
# High Level Model Architecture
from IPython.display import Image

#Image(filename='1_VGtBedNuZyX9E-07gnm2Yg.png')


In [14]:
# create the model
embedding_vector_length = 64
model_cnn_lstm = Sequential()

# Students will be starting their code from here:

# Write the code for LSTM Based Classifcation
# Embedding layer
# Convolution-1D Layer : You are free to choose the hyperparameters and the number of layers
# LSTM Layer : You are free to choose the hyperparameters and the number of layers
# Dense Layer
# Use appropriate activation function in respective layers


# Students will be ending their code here

model_cnn_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_cnn_lstm.summary())

# Change the number of epochs and the batch size depending on the RAM Size

model_cnn_lstm.fit(X_train, y_train, epochs=5, batch_size=64, verbose=1, validation_data=(X_cv, y_cv))


ValueError: ignored

In [ ]:
# Final evaluation of the CNN + RNN model using the test data
# Students will be starting their code from here:


In [ ]:
# Identify and decode 5 of the sentences misclassified by LSTM model which  were correctly classified by CNN + LSTM model
# Students will be starting their code from here:
